In [1]:
import sys
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import cmdstanpy

sys.path.append("/Users/dddd1007/project2git/cognitive_control_model/scripts/bayesian_learner_model")

import optlearner
import pandas as pd

In [2]:
all_sub_data = pd.read_csv("/Users/dddd1007/project2git/cognitive_control_model/data/input/all_data_with_2a1d1CCC_2a1d1e.csv")

## 计算 Waskom 的模型

In [3]:
v_learner = optlearner.VolatilityLearner()
s_learner = optlearner.ProbabilityLearner()

In [34]:
# 计算抽象概念的 AIC 值
AIC_ab_list = []
sub_list = np.unique(all_sub_data['Subject_num'].values)
for i in sub_list:
    temp_sub = all_sub_data[all_sub_data['Subject_num'] == i]
    env = [1 if x == 'con' else 0 for x in temp_sub['congruency']]
    v_learner.reset()
    v_learner.fit(env)
    y = temp_sub['RT']
    temp_sub['miniblock'] = temp_sub["block"].astype(str) + temp_sub["run"].astype(str)
    mini_block = pd.get_dummies(temp_sub[['miniblock']], columns= ['miniblock'])
    mini_block['p_hats'] = v_learner.p_hats
    print(mini_block.values)

    model = sm.OLS(y, mini_block.values).fit()
    AIC_ab_list.append(model.aic)

/Users/dddd1007/miniforge3/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


[[1.         0.         0.         ... 0.         0.         0.34523047]
 [1.         0.         0.         ... 0.         0.         0.51290971]
 [1.         0.         0.         ... 0.         0.         0.6084449 ]
 ...
 [0.         0.         0.         ... 0.         1.         0.21637022]
 [0.         0.         0.         ... 0.         1.         0.31337136]
 [0.         0.         0.         ... 0.         1.         0.28561826]]


KeyboardInterrupt: 

In [17]:
AIC_ab_list

[11489.701567044487,
 11452.25365615366,
 11532.682033344341,
 11771.099236518208,
 12613.38020682956,
 11726.722923565814,
 12335.12124419807,
 11737.763952779622,
 12436.835253758725,
 12670.7098200604,
 12902.64231597094,
 12792.658547724808,
 11645.318224035624,
 11960.750731006983,
 12106.937988018475,
 12125.829438619341,
 10523.859270325205,
 12216.492361509621,
 11953.488803682298,
 12686.199658106489,
 12255.583228341187,
 12423.36992458637,
 12341.395506942195,
 10982.461274562884,
 10805.235351862822,
 10801.779507851852,
 11840.167271501989,
 12656.11651805533,
 12036.038035637692,
 11927.09777274356,
 10703.261637618392,
 11098.272024634727,
 12191.731244976396,
 10792.161126517454]

In [7]:
# 计算抽象概念的 AIC 值
AIC_ab_list_without_volatile = []
sub_list = np.unique(all_sub_data['Subject_num'].values)
for i in sub_list:
    temp_sub = all_sub_data[all_sub_data['Subject_num'] == i]
    env = [1 if x == 'con' else 0 for x in temp_sub['congruency']]
    s_learner.reset()
    s_learner.fit(env)
    y = temp_sub['RT']
    temp_sub['miniblock'] = temp_sub["block"].astype(str) + temp_sub["run"].astype(str)
    mini_block = pd.get_dummies(temp_sub[['miniblock']], columns= ['miniblock'])
    mini_block['p_hats'] = s_learner.p_hats

    model = sm.OLS(y, mini_block.values).fit()
    AIC_ab_list_without_volatile.append(model.aic)

/Users/dddd1007/miniforge3/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [8]:
AIC_ab_list_without_volatile

[11491.271655256085,
 11447.921648122363,
 11532.390693546786,
 11772.228055638701,
 12613.334322797891,
 11725.647140751566,
 12332.715175526104,
 11737.9262575576,
 12436.715422501291,
 12670.684699595438,
 12902.898584704808,
 12796.518844946379,
 11645.795837713515,
 11960.738993471914,
 12107.857701324512,
 12125.861463300294,
 10526.843151225918,
 12216.43738461766,
 11952.929384023302,
 12686.157812687528,
 12256.076373527334,
 12423.442692824254,
 12343.856915178787,
 10982.757658356853,
 10805.919397913505,
 10801.763378313219,
 11840.186601697193,
 12656.066493466593,
 12035.966519406627,
 11927.00715510255,
 10703.447532571512,
 11100.819489926946,
 12191.874173479047,
 10791.991378028852]

In [35]:
# 计算S-R Bayesian Model 的 AIC 值
AIC_sr_list = []
sub_list = np.unique(all_sub_data['Subject_num'].values)
for i in sub_list:
    temp_sub = all_sub_data[all_sub_data['Subject_num'] == i]

    # left 
    temp_sub_left = temp_sub[temp_sub['stim_loc'] == 'left']
    env = temp_sub_left['Response']
    v_learner.reset()
    v_learner.fit(env)    
    temp_sub_left['miniblock'] = temp_sub_left["block"].astype(str) + temp_sub_left["run"].astype(str)
    mini_block_left = pd.get_dummies(temp_sub_left[['miniblock', 'Trial']], columns= ['miniblock'])
    mini_block_left['p_hats'] = v_learner.p_hats

    # right
    temp_sub_right = temp_sub[temp_sub['stim_loc'] == 'right']
    env = temp_sub_right['Response']
    v_learner.reset()
    v_learner.fit(env)
    temp_sub_right['miniblock'] = temp_sub_right["block"].astype(str) + temp_sub_right["run"].astype(str)
    mini_block_right = pd.get_dummies(temp_sub_right[['miniblock', 'Trial']], columns= ['miniblock'])
    mini_block_right['p_hats'] = v_learner.p_hats

    # calculate AIC
    df = pd.concat([temp_sub_left, temp_sub_right]).sort_values(by=['Trial'])
    y = df['RT'].values
    x = pd.concat([mini_block_left, mini_block_right]).sort_values(by=['Trial']).drop('Trial', axis= 1).values
    
    print(x)
    model = sm.OLS(y, x).fit()
    AIC_sr_list.append(model.aic)

/Users/dddd1007/miniforge3/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


[[1.         0.         0.         ... 0.         0.         0.34523047]
 [1.         0.         0.         ... 0.         0.         0.34523047]
 [1.         0.         0.         ... 0.         0.         0.27581234]
 ...
 [0.         0.         0.         ... 0.         1.         0.3544496 ]
 [0.         0.         0.         ... 0.         1.         0.65352008]
 [0.         0.         0.         ... 0.         1.         0.69706797]]
[[1.         0.         0.         ... 0.         0.         0.65476953]
 [1.         0.         0.         ... 0.         0.         0.65476953]
 [1.         0.         0.         ... 0.         0.         0.48709029]
 ...
 [0.         0.         0.         ... 0.         1.         0.2717479 ]
 [0.         0.         0.         ... 0.         1.         0.40078062]
 [0.         0.         0.         ... 0.         1.         0.67975243]]


KeyboardInterrupt: 

In [32]:
AIC_sr_list

[11514.140488595363,
 11460.411089099991,
 11524.440887458744,
 11778.027010444393,
 12599.542518739352,
 11715.110655721262,
 12326.457858991354,
 11740.155649014228,
 12439.987629572286,
 12663.334792604284,
 12903.894304673617,
 12819.975721745202,
 11636.219504661105,
 11953.661013136709,
 12107.35295325483,
 12125.198994942915,
 10504.94139707419,
 12215.13323971186,
 11942.661291940836,
 12670.929441250557,
 12246.208401758084,
 12414.498066092263,
 12327.549140061328,
 10982.75745936323,
 10809.084702469507,
 10784.056481286483,
 11835.806357600044,
 12655.213730852109,
 12033.11973453599,
 11925.55324555991,
 10697.610748546595,
 11115.669695907516,
 12191.945317432537,
 10794.428980552473]

In [10]:
AIC_sr_list_without_k = []
sub_list = np.unique(all_sub_data['Subject_num'].values)
for i in sub_list:
    temp_sub = all_sub_data[all_sub_data['Subject_num'] == i]

    # left 
    temp_sub_left = temp_sub[temp_sub['stim_loc'] == 'left']
    env = temp_sub_left['Response']
    s_learner.reset()
    s_learner.fit(env)    
    temp_sub_left['miniblock'] = temp_sub_left["block"].astype(str) + temp_sub_left["run"].astype(str)
    mini_block_left = pd.get_dummies(temp_sub_left[['miniblock', 'Trial']], columns= ['miniblock'])
    mini_block_left['p_hats'] = s_learner.p_hats

    # right
    temp_sub_right = temp_sub[temp_sub['stim_loc'] == 'right']
    env = temp_sub_right['Response']
    s_learner.reset()
    s_learner.fit(env)
    temp_sub_right['miniblock'] = temp_sub_right["block"].astype(str) + temp_sub_right["run"].astype(str)
    mini_block_right = pd.get_dummies(temp_sub_right[['miniblock', 'Trial']], columns= ['miniblock'])
    mini_block_right['p_hats'] = s_learner.p_hats

    # calculate AIC
    df = pd.concat([temp_sub_left, temp_sub_right]).sort_values(by=['Trial'])
    y = df['RT'].values
    x = pd.concat([mini_block_left, mini_block_right]).sort_values(by=['Trial']).drop('Trial', axis= 1).values
    
    model = sm.OLS(y, x).fit()
    AIC_sr_list_without_k.append(model.aic)

In [11]:
AIC_sr_list_without_k

[11514.09401749766,
 11459.513882311101,
 11525.104181433653,
 11778.018534152365,
 12599.351499634042,
 11717.336740580875,
 12327.049871424777,
 11740.201712011072,
 12439.930341114332,
 12662.470889211558,
 12904.544164836225,
 12820.218836131362,
 11635.807150030272,
 11954.906636081381,
 12106.812073151865,
 12125.173024714091,
 10504.688175261175,
 12215.42144156,
 11942.693174910853,
 12670.808579032666,
 12246.767294442041,
 12414.173829393148,
 12330.896122090406,
 10982.303452923743,
 10809.083037402372,
 10786.324536635568,
 11835.676757503448,
 12655.30769880353,
 12033.471355210306,
 11925.951641042586,
 10697.797073390055,
 11114.49208403343,
 12191.949336554235,
 10794.4402787323]

## 计算 Stan 模型

In [6]:
ab_learner_stanfile = "/Users/dddd1007/project2git/cognitive_control_model/Bayesian_Models/bayesian_learner_ab.stan"
bayesian_ab_learner = cmdstanpy.CmdStanModel(model_name='bayesian_ab_learner', stan_file=ab_learner_stanfile)

INFO:cmdstanpy:compiling stan program, exe file: /Users/dddd1007/project2git/cognitive_control_model/Bayesian_Models/bayesian_learner_ab
INFO:cmdstanpy:compiler options: stanc_options=None, cpp_options=None
INFO:cmdstanpy:compiled model file: /Users/dddd1007/project2git/cognitive_control_model/Bayesian_Models/bayesian_learner_ab


In [ ]:
AIC_ab_list_stan = []
sub_list = np.unique(all_sub_data['Subject_num'].values)
for i in sub_list:
    print('--- Sub ' + str(i) + '---')
    temp_sub = all_sub_data[all_sub_data['Subject_num'] == i]
    env = [1 if x == 'con' else 0 for x in temp_sub['congruency']]
    data = {'N':len(env), 'y':env}
    ab_learner_fit = bayesian_ab_learner.sample(chains=4, data = data)
    result = ab_learner_fit.summary()
    p_hats = result.Mean.values[result.Mean.index.str.contains('r')]
    temp_sub['miniblock'] = temp_sub["block"].astype(str) + temp_sub["run"].astype(str)
    mini_block = pd.get_dummies(temp_sub[['miniblock']], columns= ['miniblock'])
    mini_block['p_hats'] = p_hats
    rts = temp_sub['RT'].values

    model = sm.OLS(rts, mini_block.values).fit()
    AIC_ab_list_stan.append(model.aic)

In [37]:
AIC_ab_list_stan

[11498.160639852422,
 11468.070301899448,
 11534.545431761677,
 11770.592653455447,
 12615.237533096524,
 11728.072605044075,
 12339.84208342117,
 11739.526738962264,
 12440.1207934304,
 12670.525756605053,
 12904.063096265276,
 12811.038041170914,
 11646.75404984532,
 11957.329444890323,
 12106.094855146117,
 12125.862620932307,
 10531.051989267684,
 12216.037226075036,
 11954.397742098477,
 12685.606523833068,
 12256.753535531236,
 12417.37070376609,
 12345.65829740235,
 10989.711180971613,
 10809.117257745984,
 10801.41540460582,
 11839.22328751961,
 12654.637441839177,
 12035.988835341734,
 11927.11851191838,
 10702.036321467243,
 11109.225693927272,
 12188.5916050771,
 10790.360793308444]

In [3]:
sr_learner_stanfile = "/Users/dddd1007/project2git/cognitive_control_model/Bayesian_Models/bayesian_learner_sr.stan"
bayesian_sr_learner = cmdstanpy.CmdStanModel(model_name='bayesian_sr_learner', stan_file=sr_learner_stanfile)

INFO:cmdstanpy:compiling stan program, exe file: /Users/dddd1007/project2git/cognitive_control_model/Bayesian_Models/bayesian_learner_sr
INFO:cmdstanpy:compiler options: stanc_options=None, cpp_options=None
INFO:cmdstanpy:compiled model file: /Users/dddd1007/project2git/cognitive_control_model/Bayesian_Models/bayesian_learner_sr


In [4]:
sub_list = np.unique(all_sub_data['Subject_num'].values)
for i in sub_list:
    print('--- Sub ' + str(i) + '---')
    temp_sub = all_sub_data[all_sub_data['Subject_num'] == i]
    react = temp_sub['Response'].values
    space_loc = [1 if x == 'right' else 0 for x in temp_sub['stim_loc']]
    data = {'N':len(react), 'react':react, 'space_loc':space_loc}
    process_file_loc = '/Users/dddd1007/project2git/cognitive_control_model/data/output/bayesian_learner_samplers/sr/process/sub' + str(i) + '/.'
    sr_learner_fit = bayesian_sr_learner.sample(chains=4, data = data,
                                                output_dir=process_file_loc,
                                                show_progress=True)
    result = sr_learner_fit.summary()
    file_name = 'sub_' + str(i) + '_sr_learner.csv'
    result.to_csv('/Users/dddd1007/project2git/cognitive_control_model/data/output/bayesian_learner_samplers/sr_learner/' + file_name)

--- Sub 1---


Chain 1 - warmup:   0%|          | 0/1 [00:00<?, ?it/s]





Chain 1 - warmup:   0%|          | 0/2000 [00:00<?, ?it/s]




In [36]:
AIC_sr_list_stan = []
sub_list = np.unique(all_sub_data['Subject_num'].values)
for i in sub_list:
    print('--- Sub ' + str(i) + '---')
    temp_sub = all_sub_data[all_sub_data['Subject_num'] == i]
    estimated_data = pd.read_csv("/Users/dddd1007/project2git/cognitive_control_model/data/output/bayesian_learner_samplers/sr/extracted_data/sub_" + str(i) + "_sr_learner_.csv")
    p_hats = estimated_data['r_selected']
    temp_sub['miniblock'] = temp_sub["block"].astype(str) + temp_sub["run"].astype(str)
    mini_block = pd.get_dummies(temp_sub[['miniblock']], columns= ['miniblock'])
    mini_block['p_hats'] = p_hats
    rts = temp_sub['RT'].values

    model = sm.OLS(rts, mini_block.values).fit()
    AIC_sr_list_stan.append(model.aic)

--- Sub 20---


In [42]:
AIC_sr_list_stan

[11955.201928882132]